In [1]:
import os

# Install required packages
%pip install scrapy
%pip install selenium
%pip install pandas
%pip install crochet
%pip install scrapy-splash
%pip install scrapy playwright
# %scrapy crawl searcher

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached scrapy_splash-0.9.0-py2.py3-none-any.whl.metadata (34 kB)
Using cached scrapy_splash-0.9.0-py2.py3-none-any.whl (27 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\playwright.exe' -> 'c:\\Python311\\Scripts\\playwright.exe.deleteme'




  Using cached playwright-1.44.0-py3-none-win_amd64.whl.metadata (3.5 kB)
  Using cached pyee-11.1.0-py3-none-any.whl.metadata (2.8 kB)
Using cached playwright-1.44.0-py3-none-win_amd64.whl (29.7 MB)
   ---------------------------------------- 0.0/292.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/292.8 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/292.8 kB 660.6 kB/s eta 0:00:01
   --------- ----------------------------- 71.7/292.8 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 292.8/292.8 kB 2.0 MB/s eta 0:00:00
Using cached pyee-11.1.0-py3-none-any.whl (15 kB)


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

def load_driver():
    options = Options()
    options.add_argument(r"download.default_directory=C:/Downloads")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")
    # options.add_argument("--headless")
    options.add_experimental_option("prefs", {
        "profile.default_content_setting_values.notifications":1,
        "profile.default_content_setting_values.geolocation": 1,})


In [3]:
from datetime import datetime
from typing import Iterable
import scrapy
from scrapy_selenium import SeleniumRequest

class ItemFormat(scrapy.Item):
    card_name = scrapy.Field()
    # percentage_in_decks = scrapy.Field()
    # img = scrapy.Field()
    # specials = scrapy.Field()
    # price_per_kg = scrapy.Field()

class knightlySpider(scrapy.Spider):
    name = 'searcher'

    custom_settings = {
        'FEED_FORMAT': 'json',
        'FEED_URI': f'knightlyScrape_{datetime.now().strftime("%Y%m%d%H%M%S")}.json',
        "scrapy_selenium.SeleniumMiddleware": 800
    }

    def start_requests(self) -> Iterable[scrapy.Request]:
        urls = ['https://www.knightlygaming.co.za/collections/yugioh-singles?page=1']

        for url in urls:
            yield SeleniumRequest(url=url, callback=self.parse)
        return super().start_requests()

    def parse(self, response):

        # next_page_link = response.css("div.pag_next::attr(href)").get()
        # if next_page_link:
        #     yield scrapy.Request(url=next_page_link, meta={'playwright': True}, callback=self.parse)

        for i, cards in enumerate(response.css('li.grid__item.col-lg-4.col-md-6.col-sm-6.col-xs-6')):
            cards_item = ItemFormat()

            cards_item['card_name'] = cards.css('div.h4.grid-view-item__title::text').get(i)
            # cards_item['percentage_in_decks'] = cards.css('div.CardLabel_label__iAM7T::text').get().replace("\n", " || synergy ")
            # quote_item['img'] = quote.css('a img.CardImage_border__OcVcj.shadow::attr(src)').get()

            yield cards_item

In [4]:
from scrapy.crawler import CrawlerProcess


process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(knightlySpider)
process.start()

2024-06-04 17:54:25 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2024-06-04 17:54:25 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 3.1.3 19 Sep 2023), cryptography 41.0.4, Platform Windows-10-10.0.19045-SP0
2024-06-04 17:54:25 [scrapy.addons] INFO: Enabled addons:
[]
2024-06-04 17:54:25 [py.warnings] WARNING: C:\Users\josia\AppData\Roaming\Python\Python311\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the 